In [1]:
# This notebook is only for testing purposes.
# Here we create an example PINN with maximum parameters (num neurons, num layers, batch size...), and 
# train it to check wether the machine can hold the experiments it must do during the grid search

# Keep in mind the grid search may train multiple PINNs at the same time, so if you are using that option,
# the testing done in this notebook may not be reliable.

# Imports  + Load rod equation object

In [17]:
import numpy as np
import scipy

In [4]:
# To load this object, we must simply run the notebook containing it:
%run rod_EQS.ipynb

# Maximum parameters

In [5]:
# Max Denspt
denspt = 12
# Max number of neurons per layer:
numNeurons = 100
# Max number of layers:
numLayers = 50
# Maximum Batch size:
batch_size = 25000

# Other parameters do not have such a clear impact in the usage of resources. For instance, tanh and mish both rely
# on exponents to perform calculations, so they use more resources than ReLU, but it is not clear which uses more.
# Therefore, they are selected based on experience.

# Activation function
activator = 'tanh'
# Loss function
loss = 'mse'
# Optimizer
optimizer = 'adam'
# Initial learning rate
initial_lr = 1e-3

# Building of the testing and training grid:


In [11]:
test_gridObj = SEQ.Grid(2,{'x':0,'t':1},[[[0,2],[0,10]]],12)
train_gridObj = test_gridObj

# Validation data

In [18]:
def u_n(n,x,t):
  mu_n = (2*n+1)*np.pi/4
  T_n =  np.exp(-t*(mu_n**2))
  X_n = -np.cos(mu_n*x)*2.5/(mu_n**2)
  return X_n*T_n

depth = 100

u_pred_Simb = (np.sum((np.array([u_n(n,test_gridObj.grid[:,0],test_gridObj.grid[:,1]) for n in range(depth)])),axis=0)+5)[:,None]
valData = np.concatenate((test_gridObj.grid,u_pred_Simb),axis=1)

# Building equation Functionals, PDEs, Variables and Params

In [12]:
Functionals, PDEs, variables, params = mySys.getPDEs(numNeurons,numLayers,
                                                              activator)
m = sn.SciModel(list(variables.values()),PDEs,loss,optimizer)

In [13]:
dimlist, data = mySys.evalSystem(train_gridObj)


# Training model

In [ ]:
number_of_epochs = 100
history = m.train(dimlist,data, epochs = int(number_of_epochs),verbose=1,
                        batch_size=batch_size,learning_rate=initial_lr) 

In [ ]:
# If no errors happen during training, the gridsearch will work using 1 worker.